<a href="https://colab.research.google.com/github/jonrtaylor/twitch/blob/master/latest_correlations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install numerapi
import pandas as pd
import numerapi
import datetime
import matplotlib.pyplot as plt

today = str(datetime.date.today())

api = numerapi.NumerAPI()

round_number = api.get_current_round()
if datetime.date.today().weekday() < 3:
    round_number -= 1

dfs = []
# get last 4 rounds
for _ in range(4):
  temp_df = pd.DataFrame(api.round_details(round_number))
  temp_df['round_number'] = round_number
  dfs.append(temp_df)
  round_number -= 1

df = pd.concat(dfs)

#calculate percentile rank of correlation score by round
df['percentile_rank'] = df.groupby(['round_number','date']).rank(pct=True)
#filter to today and yesterday's daily scores
df = df[(df.date == today) | (df.date + pd.Timedelta(days=1) == today)].copy()

names = ['arbitrage', 'arbitrage2', 'arbitrage3', 'arbitrage4', 'leverage',
         'leverage2', 'leverage3', 'culebracapital', 'culebracapital2',
         'culebracapital3', 'integration_test'] 
#replace model names above with whichever models you want!
all_data = pd.DataFrame()
for name in names:
    name_df = df[df.username == name].copy()
    all_data = all_data.append(name_df, ignore_index=True)
#calculate differences from yesterday
all_data[['corr_diff', 'percentile_diff']] = all_data.sort_values(by='date').groupby(['round_number','username']).diff()[['correlation', 'percentile_rank']]
#replace with only data from today
all_data = all_data[(all_data.date == today)].copy()

2021-01-23 16:52:32,160 INFO numexpr.utils: NumExpr defaulting to 2 threads.


In [2]:
# view mean of all models per round
all_data.sort_values(by=['round_number', 'correlation'], ascending=False).groupby('round_number')[['correlation','percentile_rank','corr_diff','percentile_diff']].mean()

,correlation,percentile_rank,corr_diff,percentile_diff
round_number,,,,
244,0.000683,0.161174,0.001787,-0.004487
245,0.009996,0.227824,0.009975,0.026515
246,-0.000329,0.246524,0.002106,-0.057500
247,0.010536,0.249800,0.014080,0.031906


In [3]:
# view raw stats of all models per round
all_data.sort_values(by=['round_number', 'correlation'], ascending=False).reset_index(drop=True)

,correlation,date,username,round_number,percentile_rank,corr_diff,percentile_diff
0,0.037022,2021-01-23,integration_test,247,0.731712,0.025153,0.171047
1,0.027902,2021-01-23,leverage3,247,0.523937,0.022188,0.133288
2,0.017320,2021-01-23,arbitrage,247,0.332823,0.015727,0.044956
3,0.016489,2021-01-23,arbitrage2,247,0.322099,0.017256,0.081759
4,0.008419,2021-01-23,leverage,247,0.188051,0.016989,0.070972
5,0.006160,2021-01-23,arbitrage4,247,0.163156,0.008739,-0.045886
6,0.002382,2021-01-23,culebracapital,247,0.113367,0.002020,-0.148225
7,0.002259,2021-01-23,arbitrage3,247,0.111835,0.013178,0.023349
8,0.001749,2021-01-23,culebracapital2,247,0.104558,0.014883,0.044666
9,-0.000741,2021-01-23,culebracapital3,247,0.086174,0.007041,-0.044429
